In [1]:
import numpy as np
import pandas as pd
import re
from google.colab import drive
drive.mount("/drive")

Mounted at /drive


In [2]:
df = pd.read_excel('/drive/My Drive/6SP/logs.xlsx')
df_new = df[df['Description'].notnull()] # remove rows with empty description field
df_new = df_new[df_new['Event context'].notnull()] # remove rows with empty event context field
df_new = df_new[df_new['Event name'].notnull()] # remove rows with empty event name field
df_new.head()

,Time,Event context,Component,Event name,Description,Origin
0,"28/02/22, 10:56",Course: GP106-2021 : Computing,Logs,Log report viewed,The user with id '2614' viewed the log report ...,web
1,"28/02/22, 10:56",Course: GP106-2021 : Computing,System,Course viewed,The user with id '2614' viewed the course with...,web
2,"28/02/22, 10:04",Course: GP106-2021 : Computing,System,Course viewed,The user with id '3740' viewed the course with...,web
3,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Course viewed,The user with id '30' viewed the course with i...,web
4,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Role assigned,The user with id '30' assigned the role with i...,web


In [3]:
def userId(desc):
    return desc.split()[4][1:-1]

In [4]:
df_new['User_id'] = df_new['Description'].apply(userId)
df_new.head()

,Time,Event context,Component,Event name,Description,Origin,User_id
0,"28/02/22, 10:56",Course: GP106-2021 : Computing,Logs,Log report viewed,The user with id '2614' viewed the log report ...,web,2614
1,"28/02/22, 10:56",Course: GP106-2021 : Computing,System,Course viewed,The user with id '2614' viewed the course with...,web,2614
2,"28/02/22, 10:04",Course: GP106-2021 : Computing,System,Course viewed,The user with id '3740' viewed the course with...,web,3740
3,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Course viewed,The user with id '30' viewed the course with i...,web,30
4,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Role assigned,The user with id '30' assigned the role with i...,web,30


In [11]:
ar=df_new.User_id.unique() #35
len(ar)

493

In [5]:
# function to find the hit
def extractHitType(eventContext,eventName):
  type = eventContext.split(":")[0]
  if (type=='Course') | (type=='File') | (type=='Page') | (type=='URL'):
    if (eventName == 'Course viewed'):
      return 'course_view'
    elif (eventName == 'Course module viewed'):
      return 'resource_view'
    else:
      return "other"
  elif (type=='Forum'):
    if (eventName == 'Course module viewed'):
      return "forum_view"
    elif (eventName == 'Discussion viewed'):
      return "discusssion_view"
    elif (eventName == 'Discussion created'):
      return "discusssion_create"
    elif (eventName == 'Post created'):
      return "post_create"
    elif (eventName == 'Post updated'):
      return "post_update"
    elif (eventName == 'Some content has been posted'):
      return "content_post"   
    else:
      return "other"
  elif(type=='Assignment') | (type=='Quiz') | (type=='Interactive content'):
    if(eventName == 'A submission has been submitted'):
      return "assign_submit" 
    elif(eventName == 'Quiz attempt submitted'):
      return "quiz_attempt"
    elif(eventName == 'mod_hvp: attempt submitted'):
      return "interactivecontent_submit"
    else:
      return "other"
  else:
    return "other"

In [6]:
# function to find the hit
def extractHits(eventContext,eventName):
  type = eventContext.split(":")[0]
  if (type=='Course') | (type=='File') | (type=='Page') | (type=='URL'):
    if (eventName == 'Course viewed') |(eventName == 'Course module viewed'):
      return 'resource'    
    else:
      return "other"
  elif (type=='Forum'):
    if (eventName == 'Course module viewed') |  (eventName == 'Discussion viewed'):
      return "forum_view"   
    elif (eventName == 'Discussion created')|(eventName == 'Post created')|(eventName == 'Post updated')|(eventName == 'Some content has been posted'):
      return "forum_participation"     
    else:
      return "other"
  elif(type=='Assignment') | (type=='Quiz') | (type=='Interactive content'):
    if(eventName == 'A submission has been submitted')|(eventName == 'Quiz attempt submitted')|(eventName == 'mod_hvp: attempt submitted'):
      return "assessment"  
    else:
      return "other"
  else:
    return "other"

In [7]:
#df_new['Hit'] = df_new.apply(lambda x:extractHitType(x['Event context'],x['Event name']), axis=1)
#df['NewCol'] = df.apply(lambda x: segmentMatch(x['TimeCol'], x['ResponseCol']), axis=1)
df_new['Hit'] = df_new.apply(lambda x:extractHits(x['Event context'],x['Event name']), axis=1)
df_new.head()

,Time,Event context,Component,Event name,Description,Origin,User_id,Hit
0,"28/02/22, 10:56",Course: GP106-2021 : Computing,Logs,Log report viewed,The user with id '2614' viewed the log report ...,web,2614,other
1,"28/02/22, 10:56",Course: GP106-2021 : Computing,System,Course viewed,The user with id '2614' viewed the course with...,web,2614,resource
2,"28/02/22, 10:04",Course: GP106-2021 : Computing,System,Course viewed,The user with id '3740' viewed the course with...,web,3740,resource
3,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Course viewed,The user with id '30' viewed the course with i...,web,30,resource
4,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Role assigned,The user with id '30' assigned the role with i...,web,30,other


In [8]:
df_new = df_new[df_new.Hit != 'other']

df_new.head()

,Time,Event context,Component,Event name,Description,Origin,User_id,Hit
1,"28/02/22, 10:56",Course: GP106-2021 : Computing,System,Course viewed,The user with id '2614' viewed the course with...,web,2614,resource
2,"28/02/22, 10:04",Course: GP106-2021 : Computing,System,Course viewed,The user with id '3740' viewed the course with...,web,3740,resource
3,"28/02/22, 09:23",Course: GP106-2021 : Computing,System,Course viewed,The user with id '30' viewed the course with i...,web,30,resource
7,"28/02/22, 09:22",Course: GP106-2021 : Computing,System,Course viewed,The user with id '30' viewed the course with i...,web,30,resource
9,"28/02/22, 09:20",Course: GP106-2021 : Computing,System,Course viewed,The user with id '30' viewed the course with i...,web,30,resource


In [9]:
#df_summary = df_new.groupby(['User_id','Hit']).size().reset_index().groupby(['User_id','Hit'])[[0]].max()
df_summary = df_new.groupby(["User_id","Hit"])["User_id"].count().reset_index(name="Count")
df_summary.head()

,User_id,Hit,Count
0,124,forum_participation,1
1,124,forum_view,13
2,124,resource,14
3,1661,assessment,2
4,1661,forum_participation,1


In [10]:
df_summary.to_csv("hits_summary.csv")

In [12]:
ar=df_summary.User_id.unique() 
len(ar)

493